In [1]:
# https://www.ptsd.va.gov/understand_tx/meds_for_ptsd.asp

In [1]:
import sdv
import pandas as pd
from random import sample, choice, randint
import random
import uuid
import numpy as np
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel  
from scipy.spatial import distance

# Get ICD and prescription tables

In [25]:
diagnosis_dim = pd.read_csv("diagnosis_source.csv")
medication_dim = pd.read_csv("medication_source.csv")

# Create Population Table

size between 200-300,  ages are chosen randomly between of 22 and 88

In [3]:
## Personal demographics table

random.seed(9)

person = [{"person_id":randint(0,9999999999),"age":randint(23,88)} for y in range(0, 300)]

person_df = pd.DataFrame(person)
bins = [22, 33, 44, 55, 66, 77, 88]
labels = ["'22-33'","'33-44'","'44-55'","'55-66'","'66-77'","'77-88'"]

person_df['age_bracket'] = pd.cut(person_df['age'], bins=bins, labels=labels)
person_df['age_bracket'] = person_df['age_bracket'].astype(str)

print("POPULATION SIZE: ",len(person)) 
print(person_df.groupby("age_bracket").count()["person_id"])

POPULATION SIZE:  300
age_bracket
'22-33'    62
'33-44'    48
'44-55'    46
'55-66'    40
'66-77'    58
'77-88'    46
Name: person_id, dtype: int64


This data defines the VASRD codes and medication. Since medicaton and VA Disability benefits are 

In [4]:
vasrd = {
    "9412": "Panic disorder and/or agoraphobia",
    "9411": "Posttraumatic stress disorder",
    "6602": "Asthma"
}

medication_ptsd = [
    "Sertraline",
    "Paroxetine"
]

medication_panic = [
    "FLUoxetine",
    "Sertraline",
    "Lorazepam"
]

medication_asthma = [
    "PredniSONE",
    "Salmeterol Xinafoate Diskus (50 mcg)",
    "Albuterol 0.083% Neb Soln",
    "Fluticasone Propionate NASAL"
]

In [5]:
random.seed(4)
benefits_dim = [{"benefit_id":randint(0,9999999999),"vasrd":vasrd_row, "vasrd desc": vasrd[vasrd_row]} for vasrd_row in vasrd]
benefits_dim = pd.DataFrame(benefits_dim)
benefits_dim

,benefit_id,vasrd,vasrd desc
0,5308786135,9412,Panic disorder and/or agoraphobia
1,9033029319,9411,Posttraumatic stress disorder
2,5996024489,6602,Asthma


In [6]:
diagnosis_list = diagnosis_dim["diagnosis_id"].tolist()
medication_list = medication_dim["drug_id"].tolist()

In [7]:
random.seed(6)

n = 4
print("3/",n," of the population has a VA Disability rating")
segment = int(len(person_df["person_id"]) - (len(person_df["person_id"])/n))

benefits_id_list = benefits_dim["benefit_id"].to_list()

benefits_df = [{"person_id": person_id, "benefit_id":benefits_id_list[choice(range(0,len(benefits_id_list)))]} for person_id in person_df["person_id"][0:segment]]
benefits_df = benefits_df + [{"person_id": person_id, "benefit_id":"None"} for person_id in person_df["person_id"][segment:]]
benefits_df = pd.DataFrame(benefits_df)
benefits_df.head(n=10)

benefits_df = benefits_df.drop_duplicates(subset = ["benefit_id", "person_id"])

benefits_df= benefits_df.merge(benefits_dim, how="left", on="benefit_id")
print(benefits_df.groupby("vasrd desc").count()[["person_id"]])
print("\nNumber of people with benefits: ",benefits_df.groupby("vasrd desc").count()[["person_id"]].values.sum())



3/ 4  of the population has a VA Disability rating
                                   person_id
vasrd desc                                  
Asthma                                    80
Panic disorder and/or agoraphobia         63
Posttraumatic stress disorder             82

Number of people with benefits:  225


## Randomly sample the segmented population

The number of benefits to assign is chosen randomly. The population to be addigned medication is randomly sampled from the population with the correlated VA Disability rating. the medication is also chosen at random from the identified list of medicaitons. The rest of the population is sampled and assigned a random medication from the `prescription` table.

In [8]:
random.seed(7)
print("Create random samples to assign medications:\n")
panic_sample_size = randint(30,40)
ptsd_sample_size = randint(60,80)
asthma_sample_size = randint(60,90)
none_sample_size = randint(20,50)
print("panic sample size: ",panic_sample_size,"\nptsd sample size: ",ptsd_sample_size, "\nasthma sample size: ",asthma_sample_size,"\nno disability sample size: ", none_sample_size)
list_of_panic_population = [choice(benefits_df.loc[benefits_df["benefit_id"] == benefits_dim["benefit_id"].to_list()[0]]["person_id"].tolist()) for x in range(0, panic_sample_size)]
list_of_ptsd_population = [choice(benefits_df.loc[benefits_df["benefit_id"] == benefits_dim["benefit_id"].to_list()[1]]["person_id"].tolist()) for x in range(0, ptsd_sample_size)]
list_of_asthma_population = [choice(benefits_df.loc[benefits_df["benefit_id"] == benefits_dim["benefit_id"].to_list()[2]]["person_id"].tolist()) for x in range(0, asthma_sample_size)]
list_of_none_population = [choice(benefits_df.loc[benefits_df["benefit_id"] == "None"]["person_id"].tolist()) for x in range(0, none_sample_size)]

Create random samples to assign medications:

panic sample size:  35 
ptsd sample size:  64 
asthma sample size:  72 
no disability sample size:  40


In [9]:
random.seed(8)

medication = [{"person_id":person_id, "drug_id":medication_dim.loc[medication_dim["drug"] == medication_ptsd[choice(range(0,len(medication_ptsd)))].lower()]["drug_id"].values[0]} for person_id in list_of_ptsd_population]+\
[{"person_id":person_id, "drug_id":medication_dim.loc[medication_dim["drug"] == medication_panic[choice(range(0,len(medication_panic)))].lower()]["drug_id"].values[0]} for person_id in list_of_panic_population]+\
[{"person_id":person_id, "drug_id":medication_dim.loc[medication_dim["drug"] == medication_asthma[choice(range(0,len(medication_asthma)))].lower()]["drug_id"].values[0]} for person_id in list_of_asthma_population]+\
[{"person_id":person_id, "drug_id":medication_dim.loc[choice(range(0,len(medication_dim)))]["drug_id"]} for person_id in list_of_none_population]

medication_df = pd.DataFrame(medication)
print("Frequency of drug in the overall population: \n")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(medication_df.merge(medication_dim, how="left", on="drug_id").groupby(["drug"]).count()["person_id"])

Frequency of drug in the overall population: 

drug
albuterol 0.083% neb soln                                              11
alteplase (catheter clearance)                                          3
amlodipine                                                              1
atropine sulfate 1%                                                     1
doxazosin                                                               1
escitalopram oxalate                                                    2
fentanyl patch                                                          5
fluoxetine                                                             16
fluticasone propionate nasal                                           21
ivermectin                                                              2
lorazepam                                                               5
metformin (glucophage)                                                  1
morphine sulfate                                            

# ICD disease table

In [10]:
icd_panic = [
    {"icd_code":"F400"},
    {"icd_code":"F4001"},
    {"icd_code":"F4000"},
    {"icd_code":"F4002"}
]
icd_ptsd = [
    {"icd_code":"F431"},
    {"icd_code":"F4312"},
    {"icd_code":"F4310"},
    {"icd_code":"F4311"}
]
icd_asthma = [
    {"icd_code":"J4599"},
    {"icd_code":"J45901"},
    {"icd_code":"J4590"},
    {"icd_code":"J45998"}
]

In [23]:
random.seed(2)
print("Create random samples to assign diagnosis\n")
diagnosis_panic_pop = randint(20,50)
diagnosis_ptsd_pop = randint(70,90)
diagnosis_asthma_pop = randint(50,60)
diagnosis_none_pop = randint(20,50)

print("panic sample size: ",diagnosis_panic_pop,"\nptsd sample size: ",diagnosis_ptsd_pop, "\nasthma sample size: ",diagnosis_asthma_pop,"\nno disability sample size: ", diagnosis_none_pop)

list_of_panic_population = [choice(benefits_df.loc[benefits_df["benefit_id"] == benefits_dim["benefit_id"].to_list()[0]]["person_id"].tolist()) for x in range(0, diagnosis_panic_pop)]
list_of_ptsd_population = [choice(benefits_df.loc[benefits_df["benefit_id"] == benefits_dim["benefit_id"].to_list()[1]]["person_id"].tolist()) for x in range(0, diagnosis_ptsd_pop)]
list_of_asthma_population = [choice(benefits_df.loc[benefits_df["benefit_id"] == benefits_dim["benefit_id"].to_list()[2]]["person_id"].tolist()) for x in range(0, diagnosis_asthma_pop)]
list_of_none_population = [choice(benefits_df.loc[benefits_df["benefit_id"] == "None"]["person_id"].tolist()) for x in range(0, diagnosis_none_pop)]

Create random samples to assign diagnosis

panic sample size:  50 
ptsd sample size:  71 
asthma sample size:  51 
no disability sample size:  22


In [26]:
random.seed(1)

diagnosis = [{"person_id":person, "diagnosis_id":diagnosis_dim.loc[diagnosis_dim["icd_code"] == icd_ptsd[choice(range(0,len(icd_ptsd)))]["icd_code"]]["diagnosis_id"].values[0]} for person in list_of_ptsd_population]+\
[{"person_id":person, "diagnosis_id":diagnosis_dim.loc[diagnosis_dim["icd_code"] == icd_panic[choice(range(0,len(icd_panic)))]["icd_code"]]["diagnosis_id"].values[0]} for person in list_of_panic_population]+\
[{"person_id":person, "diagnosis_id":diagnosis_dim.loc[diagnosis_dim["icd_code"] == icd_asthma[choice(range(0,len(icd_asthma)))]["icd_code"]]["diagnosis_id"].values[0]} for person in list_of_asthma_population]+\
[{"person_id":person, "diagnosis_id":diagnosis_dim.loc[choice(range(0,len(diagnosis_dim)))]["diagnosis_id"]} for person in list_of_none_population]

diagnosis_df = pd.DataFrame(diagnosis)
print("Frequency of diagnosis in the overall population: \n")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(diagnosis_df.merge(diagnosis_dim, how="left", on="diagnosis_id").groupby(["icd_code"]).count()["person_id"])

Frequency of diagnosis in the overall population: 

icd_code
C9480       1
F400       15
F4000      13
F4001      16
F4002      12
F431       19
F4310      13
F4311      25
F4312      15
H72819      2
J4590      13
J45901     13
J4599      14
J45998     13
N809        1
S31819D     1
S90559S     2
T8529       2
V00312S     1
V9348       2
Y36441      1
Name: person_id, dtype: int64


In [13]:
# filter medication and diagnosis dim tables so they only include data that is used in the tables.
medication_dim_out = medication_dim.loc[medication_dim["drug_id"].isin(medication_df["drug_id"])]
diagnosis_dim_out = diagnosis_dim.loc[diagnosis_dim["diagnosis_id"].isin(diagnosis_df["diagnosis_id"])]


In [14]:
person_df.to_csv("demographics_table.csv", index=False)
benefits_df.to_csv("benefits_table.csv", index=False)
medication_df.to_csv("medication_table.csv", index=False)
diagnosis_df.to_csv("diagnosis_table.csv", index=False)

diagnosis_dim_out.to_csv("diagnosis_dim.csv", index=False)
medication_dim_out.to_csv("medication_dim.csv", index=False)
benefits_dim.to_csv("benefits_dim.csv", index=False)

In [27]:
diagnosis_table = diagnosis_df.merge(diagnosis_dim, how="left", on="diagnosis_id")
medication_table = medication_df.merge(medication_dim, how="left", on="drug_id")
benefits_table = benefits_df.merge(benefits_dim, how="left", on="benefit_id")


In [28]:
merged_df = person_df.merge(diagnosis_table, how="left", on="person_id")
merged_df= merged_df.merge(medication_table, how="left", on="person_id")
merged_df = merged_df.merge(benefits_table, how="left", on="person_id")
merged_df = merged_df.replace({np.nan: "none"})

In [29]:
merged_df.drop_duplicates()

,person_id,age,age_bracket,diagnosis_id,icd_code,drug_id,drug,benefit_id,vasrd_x,vasrd desc_x,vasrd_y,vasrd desc_y
0,5898329840,40,'33-44',7791961384.0,J4599,4918823640.0,prednisone,5996024489,6602,Asthma,6602,Asthma
2,2906162014,66,'55-66',none,none,none,none,5308786135,9412,Panic disorder and/or agoraphobia,9412,Panic disorder and/or agoraphobia
3,6454399878,33,'22-33',none,none,none,none,9033029319,9411,Posttraumatic stress disorder,9411,Posttraumatic stress disorder
4,3006899318,71,'66-77',9559604349.0,F4311,none,none,9033029319,9411,Posttraumatic stress disorder,9411,Posttraumatic stress disorder
5,9317563040,80,'77-88',150164051.0,F400,none,none,5308786135,9412,Panic disorder and/or agoraphobia,9412,Panic disorder and/or agoraphobia
...,...,...,...,...,...,...,...,...,...,...,...,...
418,7096892426,37,'33-44',none,none,none,none,None,none,none,none,none
419,6513561185,67,'66-77',9625731327.0,N809,none,none,None,none,none,none,none
420,1063637697,83,'77-88',none,none,4394287386.0,nifedipine cr,None,none,none,none,none
421,3722634689,75,'66-77',none,none,none,none,None,none,none,none,none


In [30]:
df = merged_df
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print((df.groupby(['vasrd desc_x','drug']).count() / df.groupby('vasrd desc_x').count())['person_id'])

vasrd desc_x                       drug                                                               
Asthma                             albuterol 0.083% neb soln                                              0.103448
                                   fluticasone propionate nasal                                           0.189655
                                   none                                                                   0.310345
                                   prednisone                                                             0.250000
                                   salmeterol xinafoate diskus (50 mcg)                                   0.146552
Panic disorder and/or agoraphobia  fluoxetine                                                             0.188889
                                   lorazepam                                                              0.066667
                                   none                                                     

In [31]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print((df.groupby(['vasrd desc_x','icd_code']).count() / df.groupby('vasrd desc_x').count())['person_id'])

vasrd desc_x                       icd_code
Asthma                             J4590       0.146552
                                   J45901      0.112069
                                   J4599       0.137931
                                   J45998      0.146552
                                   none        0.456897
Panic disorder and/or agoraphobia  F400        0.144444
                                   F4000       0.166667
                                   F4001       0.188889
                                   F4002       0.122222
                                   none        0.377778
Posttraumatic stress disorder      F431        0.189394
                                   F4310       0.136364
                                   F4311       0.189394
                                   F4312       0.159091
                                   none        0.325758
none                               C9480       0.011765
                                   F400        0.023529
    

In [32]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print((df.groupby(['icd_code','drug']).count() / df.groupby('icd_code').count())['person_id'])

icd_code  drug                                                               
C9480     fentanyl patch                                                         1.000000
F400      fluoxetine                                                             0.133333
          none                                                                   0.666667
          sertraline                                                             0.200000
F4000     fluoxetine                                                             0.133333
          lorazepam                                                              0.066667
          none                                                                   0.600000
          sertraline                                                             0.200000
F4001     fentanyl patch                                                         0.047619
          fluoxetine                                                             0.190476
          lorazepam   

## Gaussian Synthesizer 

In [102]:
from sdv.metadata import SingleTableMetadata
metadata= {
    "columns": {
        "person_id": {
            "sdtype": "categorical"
        },
        "age": {
            "sdtype": "categorical"
        },
        "vasrd": {
            "sdtype": "categorical"
        },
        "drug": {
            "sdtype": "categorical"
        },
        "diagnosis": {
            "sdtype": "categorical"
        }
    },
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}
metadata_obj = SingleTableMetadata.load_from_dict(metadata)

In [155]:
from sdv.single_table import TVAESynthesizer

synthesizer = TVAESynthesizer(metadata_obj,epochs=500)
synthesizer.fit(merged_df)

synthetic_data = synthesizer.sample(num_rows=200)

In [156]:
synthetic_data

,person_id,age,vasrd,drug,diagnosis
0,7686347231,79,9411 Posttraumatic stress disorder,NaN,NaN
1,2155788657,23,9412 Panic disorder and/or agoraphobia,fluoxetine,F4001
2,1740462140,61,6602 Asthma,NaN,NaN
3,2155788657,73,6602 Asthma,fluoxetine,F4001
4,9313862409,41,9412 Panic disorder and/or agoraphobia,sertraline,NaN
...,...,...,...,...,...
195,7228541575,42,9412 Panic disorder and/or agoraphobia,fluoxetine,F4001
196,7791492802,79,9411 Posttraumatic stress disorder,NaN,NaN
197,8466690470,47,None,NaN,NaN
198,1233939245,34,9411 Posttraumatic stress disorder,NaN,F431


In [157]:
from sdv.evaluation.single_table import run_diagnostic, evaluate_quality
from sdv.evaluation.single_table import get_column_plot
real_data = merged_df
# 1. perform basic validity checks
diagnostic = run_diagnostic(real_data, synthetic_data, metadata_obj)

# 2. measure the statistical similarity
quality_report = evaluate_quality(real_data, synthetic_data, metadata_obj)

# 3. plot the data
fig = get_column_plot(
    real_data=real_data,
    synthetic_data=synthetic_data,
    metadata=metadata_obj,
    column_name='drug'
)

fig.show()

Generating report ...

(1/2) Evaluating Data Validity: |██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1054.80it/s]|
Data Validity Score: 100.0%

(2/2) Evaluating Data Structure: |██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 444.12it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 100.0%

Generating report ...

(1/2) Evaluating Column Shapes: |███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 908.45it/s]|
Column Shapes Score: 68.57%

(2/2) Evaluating Column Pair Trends: |█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 19.79it/s]|
Column Pair Trends Score: 44.67%

Overall Score (Average): 56.62%



In [158]:
fig = get_column_plot(
    real_data=real_data,
    synthetic_data=synthetic_data,
    metadata=metadata_obj,
    column_name='age'
)
fig.show()

In [159]:
fig = get_column_plot(
    real_data=real_data,
    synthetic_data=synthetic_data,
    metadata=metadata_obj,
    column_name='vasrd'
)
fig.show()

In [160]:

fig = get_column_plot(
    real_data=merged_df,
    synthetic_data=synthetic_data,
    metadata=metadata_obj,
    column_name='diagnosis'
)

fig.show()

In [16]:
df = synthetic_data.merge(icd,right_on="icd_code",left_on="diagnosis")
df[["icd_code","long_title"]]

NameError: name 'synthetic_data' is not defined

In [ ]:
df = synthetic_data
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print((df.groupby(['vasrd','diagnosis']).count() / df.groupby('vasrd').count())['person_id'])

In [164]:
df = synthetic_data
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print((df.groupby(['vasrd','drug']).count() / df.groupby('vasrd').count())['person_id'])

vasrd                                   drug                                    
6602 Asthma                             albuterol 0.083% neb soln                   0.040816
                                        fluoxetine                                  0.081633
                                        fluticasone propionate nasal                0.102041
                                        lorazepam                                   0.040816
                                        prednisone                                  0.367347
                                        salmeterol xinafoate diskus (50 mcg)        0.040816
                                        sertraline                                  0.020408
9411 Posttraumatic stress disorder      paroxetine                                  0.020833
                                        sertraline                                  0.354167
9412 Panic disorder and/or agoraphobia  fluoxetine                                

In [18]:
df = merged_df
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print((df.groupby(['vasrd','drug']).count() / df.groupby('vasrd').count())['person_id'])

NameError: name 'merged_df' is not defined